# FOREX TRADING BOT USING A GRU, AND A GRU-LSTM MODEL

## BY JOSEPH S. LUBINDA

We install the required libraries

In [ ]:
!mkdir patterns
!mkdir dataframes
#!pip install dataframe-sql
#!conda install ibis-framework -c conda-forge
#!pip install sql_to_ibis
!pip3 install derby
!pip3 install pyarrow 
!pip3 install pandas_datareader
!pip3 install yfinance
!pip3 install hurst

!pip3 install ipywidgets
!pip3 install python-engineio==3.9
!pip3 install python-socketio==4.4.0
!pip3 install fxcmpy

Activate Jupyter extentions

In [ ]:
!jupyter nbextension install --py widgetsnbextension --user
!jupyter nbextension enable widgetsnbextension --user --py

We import the required libraries

In [1]:
# Load the necessary packages and modules

import zipfile

from numpy import cumsum, log, polyfit, sqrt, std, subtract, array, argmax
from numpy.random import randn
import numpy as np  # useful for many scientific computing in Python

import hashlib

#import talib

import pandas as pd # primary data structure library
from pandas_datareader import data as pdr
import yfinance

%matplotlib inline
import matplotlib.pyplot as plt

import sklearn
import sklearn.preprocessing
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GRU, LSTM, Bidirectional, Dense, Dropout
import tensorflow.keras as keras

import math
import datetime
import os

import fxcmpy

Create function for saving dataset to a parquet file

In [2]:
def saveDataframeToParquet(new_series_df,new_name,drive_dir='./'):
    pingInfoFilePath = drive_dir+"dataframes/"+new_name+".parquet"
    new_series_df.to_parquet(pingInfoFilePath,engine='pyarrow')

Create function for reading saved parquet data to a dataframe

In [3]:
def readSavedDataframes(new_name,drive_dir='./'):
    pingInfoFilePath = drive_dir+"dataframes/"+new_name+".parquet"
    try:
        with open(pingInfoFilePath) as f:
            return pd.read_parquet(pingInfoFilePath, engine='pyarrow')
    except IOError:
        return 'nil'

Initiate Scaler

In [4]:
sc = StandardScaler()

def adj_r2_score(r2, n, k):
    return 1-((1-r2)*((n-1)/(n-k-1)))

Import Data from CSV file

In [ ]:
df = pd.read_csv('eurusd_minute.csv')

df.tail()

Create Date-Time function

In [5]:
def CombineDateAndTime(df,date_col,time_col,timestamp_col,datetime_col):
    df[timestamp_col] = df[date_col]+" "+df[time_col]
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df[datetime_col] = df[timestamp_col]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(timestamp_col, ascending=True)
    return df

Create function for setting timestamp as DataFrame Index column

In [6]:
def setTimestampAsDFIndex(df,timestamp_col):
    df = df.set_index(timestamp_col)
    return df

Ste values for necessary variables

In [7]:
date_col ='Date'
time_col='Time'
timestamp_col='timestamp'
datetime_col='DateTime'

In [ ]:

len = df.shape[0]
data_end = len-1

years_for_data = 1
data_start = data_end-(525960*years_for_data) # 525960mins = 1 year, 1051920mins = 2 years, 1577880mins = 3 years, 2103840mins = 4 years, 2629800mins = 5 years

print(len)
print(data_start)
print(data_end)

Limit Data to the selected number of years

In [ ]:
#df_new = df[data_start:data_end]
df_new = df
df_new.tail()

Set Date-Tame columns on DataFrame

In [ ]:
df_new = CombineDateAndTime(df_new,date_col,time_col,timestamp_col,datetime_col)
df_new.tail()

Remove 2020 data from dataset (the dataset only has 4 months worth of 2020 data)

In [ ]:
df_new2 = df_new[(df_new['DateTime'] < '2020-01-01')]
df_new2.tail()

Import 2020 to 2021 data

In [ ]:
df2020_2021 = pd.read_csv('EURUSD_M1_2020-2021.csv')
df2020_2021.head()

Assign the correct BCh column values

In [ ]:
df2020_2021["BCh"] = df2020_2021["BC"]-df2020_2021["BO"]
df2020_2021.tail()

In [8]:
def setTimeColumns(df,timestamp_col,char_list,excluded_chars=[' ']):
    
    df["timestring"] = df[timestamp_col]
    a = -1
    for charGroup in char_list:
        a = a+1
        
        if charGroup[0] in excluded_chars:
            df["timestring"] = df['timestring'].str.slice(start=1)
        else:
            df[charGroup[0]] = df["timestring"].str.slice(stop=charGroup[1])
            df['timestring'] = df['timestring'].str.slice(start=charGroup[1])
    
        print(df.head())
        
    df = df.drop(columns=['timestring'])
    
    return df


Create Date - Time columns

In [9]:
timestamp_col = 'timestamp'
date_col = 'Date'
time_col = 'Time'
dateTime_col = 'DateTime'
year_col = 'year'
month_col = 'month'
day_col = 'day'
hour_col = 'hour'
minute_col = 'minute'
second_col = 'second'
char_list = [[year_col,4],[month_col,2],[day_col,2],[' ',1],[hour_col,2],[minute_col,2],[second_col,2]]

In [ ]:
df2020_2021 = setTimeColumns(df2020_2021,timestamp_col,char_list)

Create function for the creation of the **DateTime** and **timestamp** columns. 

In [10]:
def setDateTimeColumns(df,timestamp_col,date_col,time_col,dateTime_col,year_col,month_col,day_col,hour_col,minute_col,second_col):
    df[date_col] = df[year_col]+"-"+df[month_col]+"-"+df[day_col]
    df[time_col] = df[hour_col]+":"+df[minute_col]+":"+df[second_col]
    df[dateTime_col] = df[date_col]+" "+df[time_col]
    
    print(df.head())

    df[date_col] = pd.to_datetime(df[date_col])
    df[dateTime_col] = pd.to_datetime(df[dateTime_col])
    df[timestamp_col] = df[dateTime_col]
    df = df.drop(columns=[year_col,month_col,day_col,hour_col,minute_col,second_col])
    return df

Create **DateTime** and **timestamp** columns

In [ ]:
df2020_2021 = setDateTimeColumns(df2020_2021,timestamp_col,date_col,time_col,dateTime_col,year_col,month_col,day_col,hour_col,minute_col,second_col)
df2020_2021.tail()

Combine 2020 to 2021 data with initial dataset

In [ ]:
df_full = df_new2.append(df2020_2021)
df_full.head()

Drop all other unnecessary columns

In [ ]:
columns_to_drop = ['AO','AH','AL','AC','ACh']
df_full = df_full.drop(columns=columns_to_drop)
df_full.tail()

Save full Dataframe to parquet file so that you won't have to do the previous steps again next time around.

In [11]:
dataframe_parquet_file_name = 'EURUSD_2005_to_2021_dataset'

In [ ]:
saveDataframeToParquet(df_full, dataframe_parquet_file_name)

In [12]:
df_full = readSavedDataframes(dataframe_parquet_file_name)

Set timestamp as DataFrame Index

In [13]:
df_full = setTimestampAsDFIndex(df_full,timestamp_col)
df_full.tail()

,Date,Time,BO,BH,BL,BC,BCh,DateTime
timestamp,,,,,,,,
2021-04-09 16:54:00,2021-04-09,16:54:00,1.19010,1.19010,1.19006,1.19006,-0.00004,2021-04-09 16:54:00
2021-04-09 16:55:00,2021-04-09,16:55:00,1.19010,1.19011,1.19004,1.19011,0.00001,2021-04-09 16:55:00
2021-04-09 16:56:00,2021-04-09,16:56:00,1.19010,1.19015,1.19009,1.19010,0.00000,2021-04-09 16:56:00
2021-04-09 16:57:00,2021-04-09,16:57:00,1.19011,1.19017,1.19010,1.19015,0.00004,2021-04-09 16:57:00
2021-04-09 16:58:00,2021-04-09,16:58:00,1.19013,1.19013,1.18979,1.18992,-0.00021,2021-04-09 16:58:00


Create Diff column by subtracting the previous value of BC column from the current BC value

In [14]:
df_full["Diff"] = df_full.BC.diff()
df_full.tail()

,Date,Time,BO,BH,BL,BC,BCh,DateTime,Diff
timestamp,,,,,,,,,
2021-04-09 16:54:00,2021-04-09,16:54:00,1.19010,1.19010,1.19006,1.19006,-0.00004,2021-04-09 16:54:00,-0.00005
2021-04-09 16:55:00,2021-04-09,16:55:00,1.19010,1.19011,1.19004,1.19011,0.00001,2021-04-09 16:55:00,0.00005
2021-04-09 16:56:00,2021-04-09,16:56:00,1.19010,1.19015,1.19009,1.19010,0.00000,2021-04-09 16:56:00,-0.00001
2021-04-09 16:57:00,2021-04-09,16:57:00,1.19011,1.19017,1.19010,1.19015,0.00004,2021-04-09 16:57:00,0.00005
2021-04-09 16:58:00,2021-04-09,16:58:00,1.19013,1.19013,1.18979,1.18992,-0.00021,2021-04-09 16:58:00,-0.00023


Create 5 minute moving average column MA5

In [15]:
df_full['MA5'] = df_full['BC'].rolling(5).mean()
df_full['MA20'] = df_full['BC'].rolling(20).mean()
df_full['MA50'] = df_full['BC'].rolling(50).mean()
df_full['MA100'] = df_full['BC'].rolling(100).mean()
df_full["MA5Diff"] = df_full.MA5.diff()
df_full["MA20Diff"] = df_full.MA20.diff()
df_full["MA50Diff"] = df_full.MA50.diff()
df_full["MA100Diff"] = df_full.MA100.diff()
df_full.tail()

,Date,Time,BO,BH,BL,BC,BCh,DateTime,Diff,MA5,MA20,MA50,MA100,MA5Diff,MA20Diff,MA50Diff,MA100Diff
timestamp,,,,,,,,,,,,,,,,,
2021-04-09 16:54:00,2021-04-09,16:54:00,1.19010,1.19010,1.19006,1.19006,-0.00004,2021-04-09 16:54:00,-0.00005,1.190140,1.190180,1.190289,1.190319,-0.000040,-0.000012,-0.000005,-0.000003
2021-04-09 16:55:00,2021-04-09,16:55:00,1.19010,1.19011,1.19004,1.19011,0.00001,2021-04-09 16:55:00,0.00005,1.190120,1.190172,1.190285,1.190315,-0.000020,-0.000009,-0.000004,-0.000004
2021-04-09 16:56:00,2021-04-09,16:56:00,1.19010,1.19015,1.19009,1.19010,0.00000,2021-04-09 16:56:00,-0.00001,1.190106,1.190163,1.190283,1.190311,-0.000014,-0.000009,-0.000003,-0.000004
2021-04-09 16:57:00,2021-04-09,16:57:00,1.19011,1.19017,1.19010,1.19015,0.00004,2021-04-09 16:57:00,0.00005,1.190106,1.190159,1.190280,1.190309,0.000000,-0.000004,-0.000002,-0.000003
2021-04-09 16:58:00,2021-04-09,16:58:00,1.19013,1.19013,1.18979,1.18992,-0.00021,2021-04-09 16:58:00,-0.00023,1.190068,1.190143,1.190272,1.190304,-0.000038,-0.000015,-0.000008,-0.000005


In [45]:

length = df_full.shape[0]
data_end = length-1

years_for_data = 4
data_start = data_end-(525960*years_for_data) # 525960mins = 1 year, 1051920mins = 2 years, 1577880mins = 3 years, 2103840mins = 4 years, 2629800mins = 5 years

print(df_full.shape)
print(length)
print(data_start)
print(data_end)

#df_new = df_full[data_start:data_end]
df_new = df_full[data_start:data_end]
df_new.shape

(5972867, 17)
5972867
2817106
5972866


(3155760, 17)

Fit (train) the scaler

In [46]:
scDiff = sc.fit(df_new[['Diff']])
scMA5Diff = sc.fit(df_new[['MA5Diff']])
scMA20Diff = sc.fit(df_new[['MA20Diff']])
scMA50Diff = sc.fit(df_new[['MA50Diff']])
scMA100Diff = sc.fit(df_new[['MA100Diff']])

Split the data into train 80% and test 20%

In [47]:
train_size = int(df_new.shape[0]*0.8)
train_dataset, test_dataset = df_new.iloc[:train_size], df_new.iloc[train_size:]

Separate the dataset into 'BC','MA5','MA5Diff','BullActivity' and 'BearActivity'

In [48]:
df_train_Diff = train_dataset[["Diff"]].copy()
df_train_MA5Diff = train_dataset[["MA5Diff"]].copy()
df_train_MA20Diff = train_dataset[["MA20Diff"]].copy()
df_train_MA50Diff = train_dataset[["MA50Diff"]].copy()
df_train_MA100Diff = train_dataset[["MA100Diff"]].copy()

df_test_Diff = test_dataset[["Diff"]].copy()
df_test_MA5Diff = test_dataset[["MA5Diff"]].copy()
df_test_MA20Diff = test_dataset[["MA20Diff"]].copy()
df_test_MA50Diff = test_dataset[["MA50Diff"]].copy()
df_test_MA100Diff = test_dataset[["MA100Diff"]].copy()

Replace NaN with zeros

In [49]:
df_train_Diff["Diff"] = df_train_Diff["Diff"].fillna(0)
df_train_MA5Diff["MA5Diff"] = df_train_MA5Diff["MA5Diff"].fillna(0)
df_train_MA20Diff["MA20Diff"] = df_train_MA20Diff["MA20Diff"].fillna(0)
df_train_MA50Diff["MA50Diff"] = df_train_MA50Diff["MA50Diff"].fillna(0)
df_train_MA100Diff["MA100Diff"] = df_train_MA100Diff["MA100Diff"].fillna(0)

df_train_Diff["Diff"] = df_train_Diff["Diff"].fillna(0)
df_train_MA5Diff["MA5Diff"] = df_train_MA5Diff["MA5Diff"].fillna(0)
df_train_MA20Diff["MA20Diff"] = df_train_MA20Diff["MA20Diff"].fillna(0)
df_train_MA50Diff["MA50Diff"] = df_train_MA50Diff["MA50Diff"].fillna(0)
df_train_MA100Diff["MA100Diff"] = df_train_MA100Diff["MA100Diff"].fillna(0)

Scale the datasets

In [50]:
df_train_Diff_scaled = scDiff.transform(df_train_Diff)
df_train_MA5Diff_scaled = scMA5Diff.transform(df_train_MA5Diff)
df_train_MA20Diff_scaled = scMA20Diff.transform(df_train_MA20Diff)
df_train_MA50Diff_scaled = scMA50Diff.transform(df_train_MA50Diff)
df_train_MA100Diff_scaled = scMA100Diff.transform(df_train_MA100Diff)

df_test_Diff_scaled = scDiff.transform(df_test_Diff)
df_test_MA5Diff_scaled = scMA5Diff.transform(df_test_MA5Diff)
df_test_MA20Diff_scaled = scMA20Diff.transform(df_test_MA20Diff)
df_test_MA50Diff_scaled = scMA50Diff.transform(df_test_MA50Diff)
df_test_MA100Diff_scaled = scMA100Diff.transform(df_test_MA100Diff)

Create function for creating input dataset

In [51]:
def create_dataset (X, look_back = 1):
    Xs, ys = [], []

    num = X.size
 
    for i in range(num-look_back):
        v = X[i:i+look_back]
        Xs.append(v)
        ys.append(X[i+look_back])
 
    return np.array(Xs), np.array(ys)

Create input datasets

In [52]:
LOOK_BACK = 30
x_train_Diff, y_train_Diff = create_dataset(df_train_Diff_scaled,LOOK_BACK)
x_train_MA5Diff, y_train_MA5Diff = create_dataset(df_train_MA5Diff_scaled,LOOK_BACK)
x_train_MA20Diff, y_train_MA20Diff = create_dataset(df_train_MA20Diff_scaled,LOOK_BACK)
x_train_MA50Diff, y_train_MA50Diff = create_dataset(df_train_MA50Diff_scaled,LOOK_BACK)
x_train_MA100Diff, y_train_MA100Diff = create_dataset(df_train_MA100Diff_scaled,LOOK_BACK)

x_test_Diff, y_test_Diff = create_dataset(df_test_Diff_scaled,LOOK_BACK)
x_test_MA5Diff, y_test_MA5Diff = create_dataset(df_test_MA5Diff_scaled,LOOK_BACK)
x_test_MA20Diff, y_test_MA20Diff = create_dataset(df_test_MA20Diff_scaled,LOOK_BACK)
x_test_MA50Diff, y_test_MA50Diff = create_dataset(df_test_MA50Diff_scaled,LOOK_BACK)
x_test_MA100Diff, y_test_MA100Diff = create_dataset(df_test_MA100Diff_scaled,LOOK_BACK)

# Print data shape
print('x_train_Diff.shape = ',x_train_Diff.shape)
print('y_train_Diff.shape = ',y_train_Diff.shape)
print('x_train_MA5Diff.shape = ',x_train_MA5Diff.shape)
print('y_train_MA5Diff.shape = ',y_train_MA5Diff.shape)


print('x_test_Diff.shape = ',x_test_Diff.shape)
print('y_test_Diff.shape = ',y_test_Diff.shape)
print('x_test_MA5Diff.shape = ',x_test_MA5Diff.shape)
print('y_test_MA5Diff.shape = ',y_test_MA5Diff.shape) 

x_train_Diff.shape =  (2524578, 30, 1)
y_train_Diff.shape =  (2524578, 1)
x_train_MA5Diff.shape =  (2524578, 30, 1)
y_train_MA5Diff.shape =  (2524578, 1)
x_test_Diff.shape =  (631122, 30, 1)
y_test_Diff.shape =  (631122, 1)
x_test_MA5Diff.shape =  (631122, 30, 1)
y_test_MA5Diff.shape =  (631122, 1)


In [31]:
from numpy import asarray
from numpy import save

save('x_train_Diff.npy',x_train_Diff)
save('x_train_MA5Diff.npy',x_train_MA5Diff)
save('x_train_MA20Diff.npy',x_train_MA20Diff)
save('x_train_MA50Diff.npy',x_train_MA50Diff)
save('x_train_MA100Diff.npy',x_train_MA100Diff)

save('y_train_Diff.npy',y_train_Diff)
save('y_train_MA5Diff.npy',y_train_MA5Diff)
save('y_train_MA20Diff.npy',y_train_MA20Diff)
save('y_train_MA50Diff.npy',y_train_MA50Diff)
save('y_train_MA100Diff.npy',y_train_MA100Diff)

########  test  #######################
save('x_test_Diff.npy',x_test_Diff)
save('x_test_MA5Diff.npy',x_test_MA5Diff)
save('x_test_MA20Diff.npy',x_test_MA20Diff)
save('x_test_MA50Diff.npy',x_test_MA50Diff)
save('x_test_MA100Diff.npy',x_test_MA100Diff)

save('y_test_Diff.npy',y_test_Diff)
save('y_test_MA5Diff.npy',y_test_MA5Diff)
save('y_test_MA20Diff.npy',y_test_MA20Diff)
save('y_test_MA50Diff.npy',y_test_MA50Diff)
save('y_test_MA100Diff.npy',y_test_MA100Diff)

In [ ]:
from numpy import load

y_test_MA100Diff = load('y_test_MA100Diff.npy')

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GRU, LSTM, Bidirectional, Dense, Dropout
import tensorflow.keras as keras

Create models

In [53]:
import tensorflow.keras as keras

# Create BiLSTM model
def create_bilstm(units,X_train):
    model = Sequential()
    # Input layer
    model.add(Bidirectional(LSTM(units = units, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    # Hidden layer
    model.add(Bidirectional(LSTM(units = units)))
    model.add(Dense(1))
    #Compile model
    model.compile(optimizer='adam',loss='mse')
    return model


# Create GRU model
def create_gru(units,X_train):
    model = Sequential()
    # Input layer
    model.add(GRU(units = units, return_sequences = True, input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.2)) 
    # Hidden layer
    model.add(GRU(units = units)) 
    model.add(Dropout(0.2))
    model.add(Dense(units = 1)) 
    #Compile model
    model.compile(optimizer='adam',loss='mse')
    return model


#optimizer='adam',loss='mse'
# Create GRU-LSTM model
def create_gru_lstm(gru_units,lstm_units,X_train,optimizer,loss):
    model = Sequential()
    # Input layer
    model.add(GRU(units = gru_units, return_sequences = True, input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.2)) 

    # Hidden layer
    model.add(LSTM (units = lstm_units))
    model.add(Dropout(0.2)) 
    #model.add(GRU(units = gru_units)) 
    #model.add(Dropout(0.2))
    model.add(Dense(units = 1)) 
    #Compile model
    model.compile(optimizer=optimizer,loss=loss)
    return model


Set model variables

In [54]:
n_steps = LOOK_BACK-1
n_inputs = 4
n_neurons = 128
n_outputs = 4
n_layers = 2
learning_rate = 0.001
batch_size = 50
n_epochs = 100

my_optimizer='adam'
patience=10
verbose=1
loss='mse'

# GRU & LSTM Model Units/Neurons
gru_units = 20
#lstm_units = 256
lstm_units = 64

Create GRU-LSTM Model

In [55]:
Diff_model_gru_lstm = create_gru_lstm(gru_units,lstm_units,x_train_Diff)
Diff_model_gru_lstm.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (None, 30, 20)            1380      
_________________________________________________________________
dropout_8 (Dropout)          (None, 30, 20)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                21760     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 23,205
Trainable params: 23,205
Non-trainable params: 0
_________________________________________________________________


Create GRU Model

In [56]:
Diff_model_gru = create_gru(n_neurons,x_train_Diff)
Diff_model_gru.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 30, 128)           50304     
_________________________________________________________________
dropout_10 (Dropout)         (None, 30, 128)           0         
_________________________________________________________________
gru_8 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 149,505
Trainable params: 149,505
Non-trainable params: 0
_________________________________________________________________


Create BiLSTM Model

In [57]:
Diff_model_bilstm = create_bilstm(n_neurons,x_train_Diff)
Diff_model_bilstm.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 30, 256)           133120    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 527,617
Trainable params: 527,617
Non-trainable params: 0
_________________________________________________________________


Create function for training the model

In [58]:
def fit_model(model,X_train,y_train,epochs,batch_size,patience,optimizer,loss):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience)
    model.compile(
        optimizer=optimizer,
        loss=loss
    )
    history = model.fit(X_train, y_train, epochs = epochs,
                        validation_split = 0.3,
                        batch_size = batch_size, shuffle = False, 
                        callbacks = [early_stop])
    return history

Create function for making predictions

In [59]:
def prediction(model,scaler,x_test):
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(prediction)
    return prediction

In [62]:
gru_lstm_Diff_trained = fit_model(Diff_model_gru_lstm,x_train_Diff,y_train_Diff,n_epochs,batch_size,patience)

Epoch 1/100
35345/35345 [==============================] - 1297s 37ms/step - loss: 140.2206 - val_loss: 83.8850
Epoch 2/100
35345/35345 [==============================] - 1277s 36ms/step - loss: 140.2219 - val_loss: 83.8747
Epoch 3/100
35345/35345 [==============================] - 1279s 36ms/step - loss: 140.2164 - val_loss: 83.8656
Epoch 4/100
35345/35345 [==============================] - 1287s 36ms/step - loss: 140.2093 - val_loss: 83.8777
Epoch 5/100
35345/35345 [==============================] - 1288s 36ms/step - loss: 140.2082 - val_loss: 83.8642
Epoch 6/100
35345/35345 [==============================] - 1296s 37ms/step - loss: 140.2018 - val_loss: 83.8589
Epoch 7/100
35345/35345 [==============================] - 1291s 37ms/step - loss: 140.1983 - val_loss: 83.8620
Epoch 8/100
35345/35345 [==============================] - 1311s 37ms/step - loss: 140.2129 - val_loss: 83.8732
Epoch 9/100
35345/35345 [==============================] - 1283s 36ms/step - loss: 140.2188 - val_loss: 

In [89]:
def get_compiled_model(model,optimizer,loss,metrics):
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics,
    )
    return model

In [91]:
loss='mse'
metrics=["mean_squared_error"]
optimizer = "adam"
gru_lstm_Diff_trained_compiled = get_compiled_model(gru_lstm_Diff_trained,optimizer,loss,metrics)


AttributeError: 'History' object has no attribute 'compile'

In [84]:


gru_lstm_Diff_trained.save("gru_lstm_Diff_model.h5")

#keras.models.save_model(gru_lstm_Diff_trained, "gru_lstm_Diff_model.hdf5", overwrite=True, include_optimizer=True, save_format=None, save_traces=True)

Train the GRU Model on Diff data

In [ ]:
gru_Diff_trained = fit_model(Diff_model_gru,x_train_Diff,y_train_Diff,n_epochs,batch_size,patience)
gru_Diff_trained.save("gru_Diff_model")

Epoch 1/100
47276/66896 [====================>.........] - ETA: 28:38 - loss: 110.1755

Train the GRU Model on MA5Diff data

In [ ]:
gru_MA5Diff_trained = fit_model(model_gru,x_train_MA5Diff,y_train_MA5Diff,n_epochs,batch_size,patience)
gru_MA5Diff_trained.save("gru_MA5Diff_model")

Test the Diff-trained GRU model on Diff data

In [ ]:
prediction_gru_Diff_trained = prediction(gru_Diff_trained,sc,x_test_Diff)


Test the MA5Diff-trained GRU model on Diff data

In [ ]:
prediction_gru_MA5Diff_trained = prediction(gru_MA5Diff_trained,sc,x_test_Diff)